# NCSU GIS/MEA582: Geospatial Modeling and Analysis
# Section 3 - Analysis

## 3A - Global, zonal and neighborhood operations, map algebra

In [ ]:
import sys
v = sys.version_info
print(f"We are using Python {v.major}.{v.minor}.{v.micro}")

In [ ]:
!grass -c -e ~/grassdata/nc_spm_08_grass7/HW_Analysis_3B

In [ ]:
# Import Python standard library and IPython packages we need.
import subprocess
import sys
import matplotlib.pyplot as plt
from PIL import Image


# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
session = gj.init("~/grassdata/nc_spm_08_grass7/HW_Analysis_3B")


## Buffers

### Find developed areas close to lakes

Set region, create buffers: 

In [ ]:
%%bash
g.region swwake_30m -p
r.buffer lakes output=lakes_buff distances=60,120,240,500

In [ ]:
# Create Map instance
lakes_buff_map = gj.Map(filename="outputs/lakes_buff.png")
lakes_buff_map.d_rast(map="lakes_buff")

# Display map
lakes_buff_map.show()

List categories in land use map to identify category numbers for developed areas. Then run [r.mapcalc](https://grass.osgeo.org/grass76/manuals/r.mapcalc.html) to extract the developed areas within the buffers and use [r.support](https://grass.osgeo.org/grass76/manuals/r.support.html) to assign the labels from the original buffer raster to the new developed buffer raster map (needed for legend): 

In [ ]:
%%bash
r.category landuse96_28m
r.mapcalc "developed_lake = if(landuse96_28m==1 || landuse96_28m==2, lakes_buff, null())"
r.support developed_lake raster=lakes_buff
r.category developed_lake

Display results

In [ ]:
# Create Map instance
mylakesbuffmap_map = gj.Map(filename="outputs/mylakesbuffmap.png")
mylakesbuffmap_map.d_rast(map="developed_lake")
mylakesbuffmap_map.d_vect(map="streets_wake", color="grey")
mylakesbuffmap_map.d_rast(map="lakes")
mylakesbuffmap_map.d_legend(raster="developed_lake", use=[2,3,4,5], at=[5,25,2,5], flags="c")


# Display map
mylakesbuffmap_map.show()

Find the total area within buffers and the developed area in ha: 

In [ ]:
%%bash
r.report -n lakes_buff units=h
r.report -n developed_lake units=h

### Find developed areas impacted by noise from highways

Set region and create buffers along major roads. \
Intersect developed areas from landuse map with road buffers. \
Transfer the category labels and compute the affected area. \
Run r.category from GUI and browse to the txt file, or run r.category command (assuming noise_cats.txt is in your working directory).

In [ ]:
%%bash
g.region raster=landuse96_28m -p
r.buffer roadsmajor output=roads_buffers distances=250,500,2500
r.mapcalc "noise = if(landuse96_28m==1 || landuse96_28m==2, roads_buffers, null())"
r.colors noise color=ryg
r.category noise rules=noise_cats.txt separator=:
r.report -n noise units=p,h

In [ ]:
# Create Map instance
noise_map = gj.Map(filename="outputs/noise.png")
noise_map.d_rast(map="noise")
noise_map.d_vect(map="streets_wake", color="grey")
noise_map.d_legend(raster="noise", at=[5,25,2,5], flags="c")

# Display map
noise_map.show()

### Find schools affected by high levels of noise

Convert schools to raster using CORECAPACI attribute (school capacity). \
Use map algebra to overlay with noise impact buffers and compute the number of students exposed to noise (see result of [r.univar](https://grass.osgeo.org/grass76/manuals/r.univar.html)).

In [ ]:
%%bash
v.to.rast schools_wake output=schoolscap_10m use=attr attrcolumn=CORECAPACI type=point
r.mapcalc "schools_noise = if(int(schoolscap_10m) && roads_buffers == 2, int(schoolscap_10m), null())"
r.univar schools_noise
r.to.vect schools_noise output=schools_noise type=point

In [ ]:
# Create Map instance
mynoisemap_map = gj.Map(filename="outputs/mynoisemap.png")
mynoisemap_map.d_rast(map="noise")
mynoisemap_map.d_vect(map="streets_wake", color="grey")
mynoisemap_map.d_legend(raster="noise", at=[5,25,2,5], flags="c")
mynoisemap_map.d_vect(map="schools_wake", icon="basic/circle", size=10, fill_color="blue")
mynoisemap_map.d_vect(map="schools_noise", icon="basic/circle", size=14, fill_color="cyan", color="black")

# Display map
mynoisemap_map.show()

## Cost surfaces

### Compute cumulative cost surface to a given accident site based on speed limits

Use 5mi/hr speed limit for off-road areas (nulls). \
For legend set the discrete speed limit values in GUI legend dialog: \
tab Required > streets_speed, Subset > List of discrete category numbers 

In [ ]:
%%bash
g.region swwake_30m -p
v.info -c streets_wake
v.to.rast streets_wake output=streets_speedtmp use=attr attrcolumn=SPEED type=line
r.mapcalc "streets_speed = if(isnull(streets_speedtmp),5,streets_speedtmp)"
r.info streets_speed
r.colors streets_speed color=gyr

In [ ]:
# Create Map instance
streets_speed_map = gj.Map(filename="outputs/streets_speed.png")
streets_speed_map.d_rast(map="streets_speed")
streets_speed_map.d_legend(raster="streets_speed", at=[5,40,2,5], use=[5,25,35,45,65])

# Display map
streets_speed_map.show()

 Import the accident point location from GeoJSON file fire_pt.json \
Run v.in.ogr from GUI, use file browser to provide full path to the file, or run the command assuming the file is in your working directory. 

In [ ]:
!v.in.ogr input=inputs/fire_pt.json output=fire_pt

Assign travel time to cross a 30m grid cell in hours.\
Note that cost in GRASS GIS is defined as travel time per cell.\

Compute cumulative cost surface to the given point.\
**Where does the 0.018641 constant come from?** You can modify the expression to get time in minutes. 

In [ ]:
%%bash
r.mapcalc "streets_travtime = 0.018641/streets_speed"
r.cost -k streets_travtime output=streets_cost start_points=fire_pt

Compute isochrones and display the cumulative cost surface map. 

In [ ]:
%%bash
r.lake elevation=above_stream water_level=5 lake=flood seed=streams

In [ ]:
# Create Map instance
mycostmap = gj.Map(filename="outputs/mycostmap.png")
mycostmap.d_rast(map="streets_cost")
mycostmap.d_vect(map="fire_pt", color="red", icon="basic/marker", size=20)
mycostmap.d_vect(map="streets_cost_04")
mycostmap.d_legend(raster="streets_cost", at=[5,50,2,5])

# Display map
mycostmap.show()

### Find cost (travel time) from selected firestations

First make your own copy of the firestations map and list attributes.\
Then query the cumulative cost surface at the firestations location.\
The travel time in hours will be stored in the attribute column CVLAG. 

In [ ]:
%%bash
g.copy vector=firestations,myfirestations
v.info -c myfirestations
v.what.rast myfirestations raster=streets_cost column=CVLAG

Add myfirestations to Layer Manager and right click on Show attribute data.\
Click on CVLAG to order them and find the lowest cost (shortest time) > 0 (firestations with 0 cost are outside the region).\
You should see a table like this.\
Export firestations with traveltime less than 0.1 hr: 

In [ ]:
!v.out.ascii input=myfirestations separator=space precision=3 columns=ID,LOCATION,CVLAG where="CVLAG<0.1 AND CVLAG>0"

To get the computed time, you can also query the cumulative cost raster directly using coordinates (in this example it's Western Blvd firestation), or using GUI query tool by selecting the cost layer and clicking on the map: 

In [ ]:
!r.what map=streets_cost coordinates=635940.262,225912.796 separator=space

Find the least cost path for the two closest stations: 

In [ ]:
%%bash
r.drain -n input=streets_cost output=route_20Westernb start_coordinates=635940.3,225912.8
r.drain -n input=streets_cost output=route_52Hollyb start_coordinates=633178.2,221353.0
r.colors route_20Westernb color=grey
r.colors route_52Hollyb color=grey

In [ ]:
# Create Map instance
mylcpmap = gj.Map(filename="outputs/mylcpmap.png")
mylcpmap.d_rast(map="streets_cost")
mylcpmap.d_vect(map="fire_pt", color="red", icon="basic/marker", size=20)
mylcpmap.d_rast(map="route_20Westernb")
mylcpmap.d_rast(map="route_52Hollyb")

# Display map
mylcpmap.show()

Print the length of the path in cells (multiply by 30m to get approx. m).\
You should have the time in hr already from the cost map.\
**At what average speed [km/hr] needs the truck travel to get there in estimated time? Is the time, speed and distance realistic?**

#### Western

In [ ]:
!r.describe route_20Westernb

In [ ]:
distance_meters = 83 * 30
distance_km = distance_meters / 1000

print(f"Distance: {distance_meters} m")
print(f"Distance: {distance_km} km")

travel_time = 0.037
print(f"Travel Time: {travel_time} hrs")
print(f"Average Speed: {(distance_km / travel_time)} kmph")

one_mile_in_km = 1.60934
print(f"Average Speed: {(distance_km / travel_time) / one_mile_in_km} mph")

#### Holly

In [ ]:
!r.describe route_52Hollyb

In [ ]:
distance_meters = 131 * 30 # Number of cells time 30 m = distance
distance_km = distance_meters / 1000

print(f"Distance: {distance_meters} m")
print(f"Distance: {distance_km} km")

travel_time = 0.060
print(f"Travel Time: {travel_time} hrs")
print(f"Average Speed: {(distance_km / travel_time)} kmph")

one_mile_in_km = 1.60934
print(f"Average Speed: {(distance_km / travel_time) / one_mile_in_km} mph")

If you would like to display streets_cost surface in 3D as shown in the lecture, you need to multiply the surface by 1000 using r.mapcalc because the numbers in hours are very low.\
Then switch off all layers except for the streets_cost_1000 and change display to 3D. 

### Compute accessibility map for help in search for lost person

Create friction map based on land cover.\
Reminder: Time to time, you should remove or at least uncheck previously used map layers in the Layer Manager, so the layers are not rendered when you don't need them anymore.\
First display land cover classes: 

In [ ]:
%%bash
g.region swwake_30m -p
r.category landclass96

 Recode the landuse map to friction map using the rules in friction_rules.txt.

In [ ]:
!r.recode landclass96 out=friction rules=inputs/friction_rules.txt

Add the streets to friction map - they are missed by the landuse map. 

In [ ]:
!r.mapcalc "friction2 = if(streets_speed > 6, 0.1, friction)"
!r.colors friction2 rules=friction_color.txt

# Create Map instance
friction2pmap = gj.Map(filename="outputs/friction2.png")
friction2pmap.d_rast(map="friction2")
friction2pmap.d_legend(raster="friction2", at=[5,50,2,5])

# Display map
friction2pmap.show()

Compute the cost map and generate isochrones: 

In [ ]:
%%bash
r.walk -k elevation=elev_ned_30m friction=friction2 output=walkcost start_coordinates=635576,216485 lambda=0.5 max_cost=10000
r.contour walkcost output=walkcost step=1000

To display the input, import coordinates of the point where the lost person was last seen given in the file lostperson.txt. 

In [ ]:
!v.in.ascii input=inputs/lostperson.txt output=lostperson separator=comma

In [ ]:
!r.colors -e map=walkcost color=viridis

# Create Map instance
mywalkcostmap = gj.Map(filename="outputs/mywalkcostmap.png")
mywalkcostmap.d_rast(map="lakes")
mywalkcostmap.d_rast(map="walkcost")
mywalkcostmap.d_vect(map="streets_wake")
mywalkcostmap.d_vect(map="walkcost", color="red", where="level=6000", width=3)
mywalkcostmap.d_vect(map="lostperson", color="yellow", fill_color="yellow", icon="basic/marker", size=30)
mywalkcostmap.d_legend(raster="walkcost", at=[5,50,2,5])

# Display map
mywalkcostmap.show()

### Cost surfaces to line features

Compute the shortest distance to a major road.\
Set region, convert vector road map to raster:

In [ ]:
%%bash
g.region swwake_30m -p
v.to.rast roadsmajor output=roadsmajor use=val type=line

Compute the distance map and cost surface to highways: 

In [ ]:
%%bash
r.mapcalc "area_one = 1"
r.cost input=area_one output=dist_toroad start_rast=roadsmajor
r.mapcalc "dist_meters = dist_toroad * (ewres() + nsres())/2."
r.mapcalc "dist_class = int(dist_meters/500)"

In [ ]:
# Create Map instance
dist_roadsmaj = gj.Map(filename="outputs/dist_roadsmaj.png")
dist_roadsmaj.d_rast(map="dist_class")
dist_roadsmaj.d_vect(map="roadsmajor")

# Display map
dist_roadsmaj.show()

In [ ]:
%%bash
r.cost -k input=streets_travtime output=cdist_toroadk start_rast=roadsmajor
r.colors cdist_toroadk color=bgyr